TITLE HERE

Intro start. explain tennis rankings, how you move up and down, is 1 good or bad etc) explain a bit about the ace serve 
and double fault. what they are

explain our question. can we estimate a players ranking based on ratio of ace serve to double fault etc


descibe data. what are we measuring

read data

In [10]:
# load packages
library(tidyverse)
library(tidymodels)
# read the data 
# note: The code below will download the data into the current working directory and read it
url <- "https://drive.google.com/uc?export=download&id=1fOQ8sy_qMkQiQEAO6uFdRX4tLI8EpSTn"
download.file(url, "./atp2017-2019.csv")
player_result <- read_csv("./atp2017-2019.csv")
glimpse(player_result)

New names:
• `` -> `...1`
Rows: 6866 Columns: 50
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (16): tourney_id, tourney_name, surface, tourney_level, winner_seed, win...
dbl (34): ...1, draw_size, tourney_date, match_num, winner_id, winner_ht, wi...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 6,866
Columns: 50
$ ...1               <dbl> 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…
$ tourney_id         <chr> "2019-M020", "2019-M020", "2019-M020", "2019-M020",…
$ tourney_name       <chr> "Brisbane", "Brisbane", "Brisbane", "Brisbane", "Br…
$ surface            <chr> "Hard", "Hard", "Hard", "Hard", "Hard", "Hard", "Ha…
$ draw_size          <dbl> 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,…
$ tourney_level      <chr> "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "…
$ tourney_date       <dbl> 20181231, 20181231, 20181231, 20181231, 20181231, 2…
$ match_num          <dbl> 300, 299, 298, 297, 296, 295, 294, 293, 292, 291, 2…
$ winner_id          <dbl> 105453, 106421, 105453, 104542, 106421, 104871, 105…
$ winner_seed        <chr> "2", "4", "2", NA, "4", NA, "2", NA, "7", "4", "5",…
$ winner_entry       <chr> NA, NA, NA, "PR", NA, NA, NA, "PR", NA, NA, NA, NA,…
$ winner_name        <chr> "Kei Nishikori", "Daniil Medvedev", "Kei Nishikori"…
$ winner_hand   

clean and wrangle BREAKDOWN INTO SMALLER STEPS PROBABLY THERE IS A LOT

In [11]:
# filter year 2019, there is indeed a more formal way with more steps to do that,
# but we can simply do it this way with only one step because we don't need the
# year information later on
player_2019 <- filter(player_result, tourney_date <= 20199999
                       , tourney_date >=20190000) |>
  select(winner_id, loser_id, w_ace, l_ace, w_df, l_df, winner_rank, loser_rank)

# seperate the original data into winner and loser
player_winner <- player_2019 |>
  select(winner_id, w_ace, w_df, winner_rank) |>
  rename(id = winner_id, ace = w_ace, df = w_df, rank = winner_rank)
player_loser <- player_2019 |>
  select(loser_id, l_ace, l_df, loser_rank) |>
  rename(id = loser_id, ace = l_ace, df=l_df, rank = loser_rank)

# Combine them together by rows to form a data into a tidy format
player_clean <- bind_rows(player_winner, player_loser)
head(player_clean)

id,ace,df,rank
<dbl>,<dbl>,<dbl>,<dbl>
105815,7,2,63
111815,17,5,93
105815,9,0,63
111815,11,0,93
105526,22,3,58
105815,11,1,63


seperate training data from testing data or whatever it is called

In [15]:
# Split data into training data and testing data
player_split <- player_clean |>
  initial_split(prop = 0.75, strata = rank)
player_training <- training(player_split)
player_testing <- testing(player_split)
head(player_training)

id,ace,df,rank
<dbl>,<dbl>,<dbl>,<dbl>
104926,5,4,13
106043,7,1,19
126774,8,1,15
104925,6,2,1
105676,9,3,22
105807,20,3,23


using training data make a table that contains variable more related to what we are doing.
means of the predictor variables you plan to use in your analysis and how many rows have missing data. 


In [16]:
# Exploratory data analysis to training data
Summarised_training <- player_training |>
  summarize(rank_mean = mean(rank, na.rm = TRUE), 
            missing_id = sum(is.na(id)), 
            missing_ace = sum(is.na(ace)),
            missing_df = sum(is.na(df)),
            missing_rank = sum(is.na(rank)))
Summarised_training

rank_mean,missing_id,missing_ace,missing_df,missing_rank
<dbl>,<int>,<int>,<int>,<int>
68.20107,0,81,81,6


visualize. show a distribution of data.histogram or something of rankings of people entered and maybe ratio distributions

WRITING SECTION describe how we are going to do the regression question. How will we visualize it ?

WRITING SECTIONOutcomes. predict results. impact. questions will it lead too